In [9]:
import time
import datetime

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

SEED = 260420010

In [10]:
df = pd.read_csv('../data/3_fill_nans/processed_data.csv')
df = df.sample(frac=1, replace=False, random_state=SEED)

# One Hot Encode

In [11]:
one_hot_col = ["data_source", "COMMODITY_RELEASED_TYPE", "ON_OFF_SHORE", "LOCATION_TYPE", 
               "ITEM_INVOLVED", "MATERIAL_INVOLVED", "SYSTEM_PART_INVOLVED", 
               "INCIDENT_AREA_TYPE", "PIPE_FACILITY_TYPE", "PIPELINE_FUNCTION", "INVESTIGATION_STATUS",
               "zone", 'RELEASE_TYPE', 'CAUSE_DETAILS', 'CAUSE']
# one_hot_col = ["COMMODITY_RELEASED_TYPE", "ON_OFF_SHORE", "LOCATION_TYPE", 
#                "MATERIAL_INVOLVED", 
#                "PIPE_FACILITY_TYPE", "PIPELINE_FUNCTION",
#                "zone"]

In [12]:
test = 0

for col in one_hot_col:
    unique_features = df[col].nunique()
    test += unique_features
    print(f"Ilość unikalnych cech w kolumnie '{col}': {unique_features}")
print(test - len(one_hot_col))

Ilość unikalnych cech w kolumnie 'data_source': 5
Ilość unikalnych cech w kolumnie 'COMMODITY_RELEASED_TYPE': 10
Ilość unikalnych cech w kolumnie 'ON_OFF_SHORE': 2
Ilość unikalnych cech w kolumnie 'LOCATION_TYPE': 7
Ilość unikalnych cech w kolumnie 'ITEM_INVOLVED': 31
Ilość unikalnych cech w kolumnie 'MATERIAL_INVOLVED': 10
Ilość unikalnych cech w kolumnie 'SYSTEM_PART_INVOLVED': 21
Ilość unikalnych cech w kolumnie 'INCIDENT_AREA_TYPE': 5
Ilość unikalnych cech w kolumnie 'PIPE_FACILITY_TYPE': 8
Ilość unikalnych cech w kolumnie 'PIPELINE_FUNCTION': 11
Ilość unikalnych cech w kolumnie 'INVESTIGATION_STATUS': 5
Ilość unikalnych cech w kolumnie 'zone': 8
Ilość unikalnych cech w kolumnie 'RELEASE_TYPE': 5
Ilość unikalnych cech w kolumnie 'CAUSE_DETAILS': 54
Ilość unikalnych cech w kolumnie 'CAUSE': 10
177


In [13]:
def one_hot_encode(input_dataframe: pd.DataFrame):
    for col in one_hot_col:
        one_hot_encoded = pd.get_dummies(input_dataframe[col], dummy_na=True, prefix=col, dtype=int)
        null_cols = []
        for oh_enc_name, oh_enc_col in one_hot_encoded.items():
            if oh_enc_col.sum() == 0:
                null_cols.append(oh_enc_name)
        input_dataframe = pd.concat([input_dataframe, one_hot_encoded], axis=1)
        input_dataframe.drop(null_cols + [col], axis=1, inplace=True)
    return input_dataframe


In [14]:
df_one_hot = one_hot_encode(df)

In [15]:
nan_columns = df_one_hot.columns[df_one_hot.isna().any()].tolist()

if len(nan_columns) == 0:
        print("Nie ma wartości NaN w żadnej kolumnie.")
else:
    print("Kolumny zawierające wartości NaN:")
    print(nan_columns)

    # Obliczanie procentowego udziału wartości NaN w każdej kolumnie
    for col in nan_columns:
        nan_percentage = (df_one_hot[col].isna().sum() / len(df)) * 100
        print(f"Procent NaN w kolumnie {col}: {nan_percentage:.2f}%")

Kolumny zawierające wartości NaN:
['NARRATIVE', 'inst_age_in_days']
Procent NaN w kolumnie NARRATIVE: 0.53%
Procent NaN w kolumnie inst_age_in_days: 28.09%


In [16]:
df_one_hot.to_csv(f'../data/4_one_hot/processed_data.csv', index=False)